In [1]:
using Plots
# The GR plotting background is significantly faster
gr();

In [2]:
function v_target(t)
    return [1, sin(t*2*pi/10)]
    
    # This codes for a zigzag with period 10
    if t % 10 >= 10/2
        return [1, 1]
    else
        return [1, -1]
    end
end

function a_predator(t, k_r, p_r, k_m)
    # Acceleration is directed directly at the prey, and is scaled by the mass
    return normalize(p_r - k_r)/k_m
end

function mass_to_period(kozai_masses)
    # Euler's Method time resolution
    dt = 0.5
    time_max=1000000

    # Preallocate giant arrays and use them repeatedly for speed
    print("Allocating memory...")
    flush(STDOUT)
    kozai = zeros(Int(time_max/dt+1),2)
    y_s = zeros(time_max)
    results = []
    print(" done.\n")
    flush(STDOUT)
    
    for kozai_mass in kozai_masses
        # Set initial conditions
        p_r = [0, 0]
        k_r = [0, 0]
        k_v = [0, 0]
        
        # This is just a fairly general Euler's Method solver for parametric DEs
        for (index, t) in enumerate(0:dt:time_max)
            p_r += v_target(t) .* dt
            k_v += a_predator(t, k_r, p_r, kozai_mass) .* dt

            if norm(k_v) != 0
                k_v = normalize(k_v) .* min(norm(k_v))
            end

            k_r += k_v .* dt

            kozai[index,:] = k_r
        end
        
        # Now we "flatten" the parametric position array [(x, y)...]
        # into a x vs. y array [y1, y2 ...]
        # NOTE: This only makes sense when the parametric curve passes
        # the vertical line test, which, in many cases, it does not.
        fill(y_s, 0)
        for i in 1:size(kozai,1)
            (x, y) = kozai[i,:]
            if ceil(Int, x) > time_max
                # If, as happens by chance, the prey passes max_time,
                # truncate to fit.
                continue
            end
            y_s[ceil(Int, x)] = round(Int,y)
        end
        
        # Make sure the trajectory is centered around zero so the signal is clean
        y_s -= mean(y_s)
        
        # Now convert to frequency space using a Fast Fourier Transform
        # Lots of random constants floating around to make the frequency mappings work.
        dT = 1 # This is 1 because we are working in index space
        yf = 2.0/(time_max/dT) * abs(rfft(y_s))
        xf = linspace(0, 0.5/dT, div(time_max/dT,2))
        
        # A heuristic to extract the "low" frequency modulation. Essentially, 
        # we find the strongest signal, which is the  "high" frequency 
        # oscillation and look for a peak at less than half its frequency.
        period = 1/xf[indmax(yf[10:div(indmax(yf),2)])+9]
        push!(results, period)
        
        @printf("%d\r", kozai_mass)
        flush(STDOUT)
    end
    results
end

function polyfit(x, y, n)
    # Really simple polynomial least-squares fit
    A = [ float(x[i])^p for i = 1:length(x), p = 0:n ]
    A \ y
end

function model(coeff)
    # Produce a model closure from the polyfit output
    function result(x)
        reduce(+, (c*x^(p-1) for (p, c) in enumerate(coeff)))
    end
end;

In [3]:
p_r = [0, 0]
k_r = [0, 0]
k_v = [0, 0]

dt = 0.5

kozai_mass=140
time_max=1000000
kozai = zeros(Int(time_max/dt+1),2)

prey = []

for (index, t) in enumerate(0:dt:time_max)
    p_r += v_target(t) .* dt
    k_v += a_predator(t, k_r, p_r, kozai_mass) .* dt

    if norm(k_v) != 0
        k_v = normalize(k_v) .* min(norm(k_v))
    end

    k_r += k_v .* dt

    kozai[index,:] = k_r
    push!(prey, p_r)
end
@printf("%d frames\n", size(kozai,1))

2000001 frames


In [12]:
plot(kozai[1:min(50000,end),1], kozai[1:min(50000,end),2])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10000 
 
 
 20000 
 
 
 -50 
 
 
 0 
 
 
 50 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,198.302 30.6038,198.302 30.6039,198.3 30.604,198.296 30.6042,198.289 30.6044,198.281 30.6047,198.269 30.605,198.255 30.6053,198.238 30.6057,198.219 
 30.6061,198.197 30.6065,198.173 30.607,198.148 30.6075,198.12 30.608,198.092 30.6086,198.063 30.6092,198.033 30.6099,198.004 30.6106,197.974 30.6114,197.944 
 30.6122,197.915 30.613,197.885 30.6139,197.855 30.6148,197.825 30.6157,197.795 30.6167,197.764 30.6177,197.732 30.6188,197.699 30.6199,197.665 30.6211,197.63 
 30.6222,197.595 30.6235,197.558 30.6247,197.521 30.626,197.483 30.6274,197.445 30.6288,197.407 30.6302,197.368 30.6317,197.329 30.6332,197.291 30.6347,197.252 
 30.6363,197.214 30.638,197.175 30.6396,197.137 30.6413,197.098 30.6431,197.06 30.6449,197.021 30.6467,196.981 30.6486,196.942 30.6505,196.901 30.6524,196.86 
 30.6544,196.819 30.6565,196.777 30.6585,196.734 30.6607,196.692 30.6628,196.649 30.665,196.606 30.6672,196.562 30.6695,196.519 30.6718,196.476 30.6742,196.433 
 30.6766,196.391 30.679,196.348 30.6815,196.305 30.684,196.263 30.6866,196.22 30.6891,196.177 30.6918,196.134 30.6945,196.091 30.6972,196.047 30.6999,196.003 
 30.7027,195.958 30.7056,195.913 30.7085,195.868 30.7114,195.823 30.7143,195.777 30.7173,195.732 30.7204,195.687 30.7235,195.641 30.7266,195.596 30.7297,195.551 
 30.7329,195.506 30.7362,195.461 30.7395,195.416 30.7428,195.371 30.7461,195.326 30.7495,195.282 30.753,195.237 30.7565,195.191 30.76,195.146 30.7636,195.1 
 30.7672,195.054 30.7708,195.008 30.7745,194.962 30.7782,194.915 30.782,194.868 30.7858,194.822 30.7896,194.775 30.7935,194.729 30.7975,194.683 30.8014,194.636 
 30.8054,194.591 30.8095,194.545 30.8136,194.499 30.8177,194.453 30.8219,194.408 30.8261,194.362 30.8303,194.316 30.8346,194.27 30.839,194.224 30.8433,194.178 
 30.8477,194.132 30.8522,194.085 30.8567,194.038 30.8612,193.991 30.8658,193.945 30.8704,193.898 30.8751,193.851 30.8798,193.805 30.8845,193.758 30.8893,193.712 
 30.8941,193.666 30.8989,193.62 30.9038,193.574 30.9088,193.529 30.9137,193.483 30.9188,193.437 30.9238,193.392 30.9289,193.346 30.9341,193.3 30.9392,193.254 
 30.9445,193.208 30.9497,193.161 30.955,193.115 30.9604,193.069 30.9657,193.022 30.9712,192.976 30.9766,192.93 30.9821,192.884 30.9877,192.838 30.9932,192.792 
 30.9989,192.747 31.0045,192.702 31.0102,192.656 31.016,192.611 31.0218,192.566 31.0276,192.521 31.0334,192.476 31.0394,192.431 31.0453,192.386 31.0513,192.341 
 31.0573,192.296 31.0634,192.251 31.0695,192.205 31.0756,192.16 31.0818,192.114 31.088,192.069 31.0943,192.024 31.1006,191.979 31.107,191.934 31.1134,191.89 
 31.1198,191.845 31.1263,191.801 31.1328,191.757 31.1393,191.713 31.1459,191.669 31.1525,191.626 31.1592,191.582 31.1659,191.538 31.1727,191.494 31.1795,191.45 
 31.1863,191.407 31.1932,191.363 31.2001,191.319 31.2071,191.275 31.214,191.231 31.2211,191.187 31.2282,191.144 31.2353,191.1 31.2424,191.057 31.2496,191.014 
 31.2569,190.971 31.2641,190.928 31.2715,190.886 31.2788,190.844 31.2862,190.801 31.2937,190.759 31.3011,190.717 31.3086,190.675 31.3162,190.633 31.3238,190.591 
 31.3314,190.549 31.3391,190.507 31.3469,190.465 31.3546,190.422 31.3624,190.38 31.3703,190.339 31.3781,190.297 31.3861,190.255 31.394,190.214 31.402,190.173 
 31.4101,190.132 31.4182,190.091 31.4263,190.051 31.4344,190.011 31.4427,189.97 31.4509,189.93 31.4592,189.89 31.4675,189.85 31.4759,189.81 31.4843,189.77 
 31.4927,189.73 31.5012,189.69 31.5097,189.65 31.5183,189.611 31.5269,189.571 31.5356,189.531 31.5443,189.492 31.553,189.452 31.5618,189.413 31.5706,189.375 
 31.5794,189.336 31.5883,189.298 31.5972,189.26 31.6062,189.222 31.6152,189.184 31.6243,189.146 31.6334,189.108 31.6425,189.071 31.6517,189.033 31.6609,188.996 
 31.6701,188.958 31.6794,188.921 31.6888,188.883 31.6982,188.84

In [5]:
#x_max = Int(ceil(maximum(kozai[:,1])))
y_s = zeros(time_max)
for i in 1:size(kozai,1)
    (x, y) = kozai[i,:]
    if ceil(Int, x) > time_max
        continue
    end
    y_s[ceil(Int, x)] = round(Int,y)
end
y_s -= mean(y_s)
dT = 1
yf = 2.0/(time_max/dT) * abs(rfft(y_s))
xf = linspace(0, 0.5/dT, div(time_max/dT,2))

pks= zeros(length(xf))
pks[findmax(yf[10:div(findmax(yf)[2],2)])[2]+9] = 0.1
1/xf[findmax(yf[10:div(findmax(yf)[2],2)])[2]+9]

9615.365384615385

In [6]:
plot_crop = 1000
plot(xf[1:min(plot_crop,end)], yf[1:min(plot_crop,end)])
plot!(xf[1:min(plot_crop,end)], pks[1:min(plot_crop,end)])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00000 
 
 
 0.00025 
 
 
 0.00050 
 
 
 0.00075 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 39.4926,384.952 40.0497,349.591 40.6068,376.232 41.1639,370.349 41.7211,377.883 42.2782,372.021 42.8353,373.261 43.3925,376.399 43.9496,373.258 44.5067,367.137 
 45.0638,380.517 45.621,370.714 46.1781,376.656 46.7352,377.406 47.2923,381.181 47.8495,373.57 48.4066,379.61 48.9637,376.433 49.5209,377.372 50.078,362.468 
 50.6351,370.311 51.1922,363.589 51.7494,380.9 52.3065,364.243 52.8636,377.215 53.4208,372.055 53.9779,370.872 54.535,370.452 55.0921,379.981 55.6493,382.159 
 56.2064,370.775 56.7635,370.821 57.3206,356.055 57.8778,361.662 58.4349,379.185 58.992,375.053 59.5492,369.041 60.1063,375.519 60.6634,358.034 61.2205,373.889 
 61.7777,370.502 62.3348,380.902 62.8919,375.664 63.449,375.938 64.0062,368.002 64.5633,370.946 65.1204,363.18 65.6776,369.74 66.2347,366.293 66.7918,378.926 
 67.3489,376.784 67.9061,370.406 68.4632,370.242 69.0203,370.688 69.5775,365.093 70.1346,376.639 70.6917,376.906 71.2488,361.482 71.806,373.376 72.3631,358.858 
 72.9202,367.553 73.4773,376.738 74.0345,366.211 74.5916,370.028 75.1487,380.168 75.7059,374.188 76.263,383.177 76.8201,376.681 77.3772,373.841 77.9344,359.33 
 78.4915,378.973 79.0486,366.728 79.6057,370.881 80.1629,365.162 80.72,356.611 81.2771,355.051 81.8343,368.176 82.3914,367.132 82.9485,367.706 83.5056,372.008 
 84.0628,356.475 84.6199,361.527 85.177,351.896 85.7342,379.563 86.2913,366.324 86.8484,369.67 87.4055,370.378 87.9627,356.307 88.5198,367.751 89.0769,367.013 
 89.634,356.17 90.1912,375.02 90.7483,363.622 91.3054,375.249 91.8626,360.768 92.4197,340.504 92.9768,354.939 93.5339,351.398 94.0911,352.471 94.6482,287.998 
 95.2053,194.109 95.7624,184.816 96.3196,89.1529 96.8767,351.459 97.4338,3.93701 97.991,83.7806 98.5481,354.574 99.1052,288.835 99.6623,305.653 100.219,350.534 
 100.777,328.632 101.334,358.557 101.891,349.269 102.448,351.99 103.005,352.747 103.562,378.202 104.119,343.862 104.676,364.687 105.234,355.876 105.791,376.95 
 106.348,360.898 106.905,366.519 107.462,348.726 108.019,367.739 108.576,349.133 109.134,370.089 109.691,358.86 110.248,361.002 110.805,338.967 111.362,369.252 
 111.919,342.652 112.476,375.216 113.033,374.09 113.591,352.869 114.148,369.405 114.705,337.652 115.262,358.316 115.819,358.402 116.376,367.634 116.933,354.726 
 117.49,371.586 118.048,360.761 118.605,360.123 119.162,377.406 119.719,352.618 120.276,346.97 120.833,368.798 121.39,344.017 121.947,340.539 122.505,341.481 
 123.062,347.65 123.619,371.875 124.176,351.913 124.733,345.425 125.29,360.698 125.847,344.724 126.404,352.427 126.962,359.417 127.519,359.237 128.076,358.02 
 128.633,367.182 129.19,353.805 129.747,370.083 130.304,369.588 130.861,367.074 131.419,361.634 131.976,357.468 132.533,348.731 133.09,371.574 133.647,378.812 
 134.204,380.079 134.761,347.317 135.319,342.172 135.876,362.665 136.433,353.635 136.99,352.671 137.547,380.792 138.104,355.803 138.661,340.592 139.218,376.619 
 139.776,359.468 140.333,376.867 140.89,355.633 141.447,325.988 142.004,334.5 142.561,341.649 143.118,350.95 143.675,358.43 144.233,365.068 144.79,311.39 
 145.347,348.676 145.904,346.133 146.461,327.995 147.018,365.327 147.575,348.752 148.132,372.537 148.69,360.493 149.247,371.706 149.804,363.016 150.361,327.987 
 150.918,333.668 151.475,341.76 152.032,306.09 152.589,338.098 153.147,338.697 153.704,326.83 154.261,340.196 154.818,303.981 155.375,326.759 155.932,278.105 
 156.489,365.194 157.046,289.45 157.604,302.143 158.161,328.085 158.718,352.301 159.275,334.627 159.832,329.379 160.389,356.951 160.946,366.249 161.503,341.12 
 162.061,351.182 162.618,351.02 163.175,326.101 163.732,366.598 164.289,370.979 164.846,380.215 165.403,356.174 165.961,310.649 166.518,329.096 167.075,346.477

In [13]:
anim_dt=10
@gif for i in 1:anim_dt:4000
    @printf("%d\r", i)
    flush(STDOUT)
    plot(map(x -> x[1], prey[1:i]), map(x -> x[2], prey[1:i]))
    plot!(kozai[1:i,1], kozai[1:i,2])
end every 1

INFO: Saved animation to /home/pythonnut/Documents/prog/sipz/tmp.gif


Plots.AnimatedGif("/home/pythonnut/Documents/prog/sipz/tmp.gif")

In [18]:
X = 50:1:200
periods = mass_to_period(X);

#for x in zip(X, periods)
#    @printf("%d,%f\n", x[1], x[2])
#end

Allocating memory... done.


In [19]:
plot(X, periods)
fit = polyfit(X, periods,2)
plot!(X, map(model(fit), X))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 125 
 
 
 150 
 
 
 175 
 
 
 200 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 48.3815,357.976 52.0327,381.544 55.6839,384.952 59.3351,369.954 62.9863,368.003 66.6375,364.503 70.2887,366.252 73.9399,376.198 77.5911,366.131 81.2423,347.442 
 84.8936,357.483 88.5448,347.442 92.196,358.937 95.8472,359.251 99.4984,351.476 103.15,359.561 106.801,361.064 110.452,355.606 114.103,339.127 117.754,346.063 
 121.406,342.369 125.057,361.5 128.708,336.807 132.359,350.045 136.01,321.964 139.662,347.889 143.313,341.584 146.964,349.835 150.615,328.219 154.267,332.729 
 157.918,321.964 161.569,334.975 165.22,338.56 168.871,327.482 172.523,314.285 176.174,330.705 179.825,322.381 183.476,314.77 187.127,315.728 190.779,308.078 
 194.43,301.078 198.081,308.623 201.732,302.911 205.383,308.623 209.035,306.97 212.686,300.455 216.337,303.509 219.988,297.233 223.64,305.84 227.291,305.266 
 230.942,289.481 234.593,293.829 238.244,295.894 241.896,285.618 245.547,289.481 249.198,294.525 252.849,290.227 256.5,285.618 260.152,278.929 263.803,285.618 
 267.454,278.929 271.105,271.526 274.756,256.475 278.408,295.894 282.059,252.837 285.71,283.187 289.361,263.289 293.013,271.526 296.664,264.369 300.315,255.28 
 303.966,257.652 307.617,257.652 311.269,240.869 314.92,243.677 318.571,245.047 322.222,243.677 325.873,239.431 329.525,227.028 333.176,233.438 336.827,236.484 
 340.478,218.359 344.129,251.588 347.781,218.359 351.432,240.869 355.083,214.667 358.734,221.919 362.386,208.865 366.037,193.946 369.688,208.865 373.339,206.857 
 376.99,206.857 380.642,193.946 384.293,189.284 387.944,196.206 391.595,193.946 395.246,198.422 398.898,208.865 402.549,196.206 406.2,186.879 409.851,159.719 
 413.502,168.515 417.154,174.054 420.805,171.316 424.456,179.351 428.107,156.648 431.759,168.515 435.41,150.28 439.061,146.978 442.712,179.351 446.363,125.317 
 450.015,153.502 453.666,143.593 457.317,168.515 460.968,146.978 464.619,136.564 468.271,132.913 471.922,129.165 475.573,121.366 479.224,125.317 482.875,117.306 
 486.527,121.366 490.178,108.843 493.829,108.843 497.48,104.431 501.132,113.133 504.783,104.431 508.434,113.133 512.085,90.4021 515.736,90.4021 519.388,99.8899 
 523.039,69.6191 526.69,95.2158 530.341,85.4425 533.992,64.0043 537.644,64.0043 541.295,52.2133 544.946,58.2055 548.597,58.2055 552.248,58.2055 555.9,58.2055 
 559.551,64.0043 563.202,32.9754 566.853,52.2133 570.505,39.6092 574.156,64.0043 577.807,32.9754 581.458,46.0181 585.109,52.2133 588.761,26.1047 592.412,3.93701 
 596.063,39.6092 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 48.3815,373.193 52.0327,372.042 55.6839,370.876 59.3351,369.693 62.9863,368.494 66.6375,367.279 70.2887,366.047 73.9399,364.799 77.5911,363.535 81.2423,362.255 
 84.8936,360.959 88.5448,359.646 92.196,358.317 95.8472,356.972 99.4984,355.61 103.15,354.232 106.801,352.839 110.452,351.428 114.103,350.002 117.754,348.559 
 121.406,347.1 125.057,345.625 128.708,344.134 132.359,342.626 136.01,341.103 139.662,339.562 143.313,338.006 146.964,336.434 150.615,334.845 154.267,333.24 
 157.918,331.619 161.569,329.981 165.22,328.327 168.871,326.657 172.523,324.971 176.174,323.269 179.825,321.55 183.476,319.815 187.127,318.064 190.779,316.296 
 194.43,314.513 198.081,312.713 201.732,310.897 205.383,309.064 209.035,307.216 212.686,305.351 216.337,303.47 219.988,301.573 223.64,299.659 227.291,297.729 
 230.942,295.783 234.593,293.821 238.244,291.843 241.896,289.848 245.547,287.837 249.198,285.81 252.849,283.766 256.5,281.707 260.152,279.631 263.803,277.538 
 267.454,275.43 271.105,273.305 274.756,271.165 278.408,269.007 282.059,266.834 285.71,264.645 289.361,262.439 293.013,260.217 296.664,257.978 300.315,255.724 
 303.9

In [20]:
plot(X, periods - map(model(fit), X))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 125 
 
 
 150 
 
 
 175 
 
 
 200 
 
 
 -1000 
 
 
 0 
 
 
 1000 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 48.3815,86.751 52.0327,261.821 55.6839,294.221 59.3351,196.372 62.9863,191.052 66.6375,174.867 70.2887,195.98 73.9399,275.256 77.5911,212.912 81.2423,89.609 
 84.8936,169.913 88.5448,108.089 92.196,198.92 95.8472,210.669 99.4984,165.25 103.15,232.266 106.801,252.785 110.452,224.114 114.103,117.507 117.754,176.848 
 121.406,161.015 125.057,306.961 128.708,142.632 132.359,247.07 136.01,58.9792 139.662,253.502 143.313,219.866 146.964,289.447 150.615,147.597 154.267,190.911 
 157.918,126.151 161.569,229.894 165.22,267 168.871,200.368 172.523,118.84 176.174,247.197 179.825,200.411 183.476,158.798 187.127,177.984 190.779,136.317 
 194.43,99.3767 198.081,165.562 201.732,137.965 205.383,191.402 209.035,192.786 212.686,159.847 216.337,194.802 219.988,163.788 223.64,238.303 227.291,247.907 
 230.942,149.891 234.593,194.581 238.244,223.22 241.896,164.569 245.547,206.172 249.198,256.252 252.849,240.284 256.5,222.231 260.152,189.555 263.803,251.752 
 267.454,219.306 271.105,181.922 274.756,90.4858 278.408,384.952 282.059,95.3943 285.71,325.852 289.361,200.547 293.013,274.625 296.664,239.787 300.315,191.386 
 303.966,224.266 307.617,240.464 311.269,137.912 314.92,174.223 318.571,200.473 322.222,207.424 325.873,194.13 329.525,123.167 333.176,185.572 336.827,224.265 
 340.478,113.126 344.129,365.823 347.781,147.937 351.432,324.945 355.083,157.059 358.734,226.233 362.386,151.697 366.037,64.0688 369.688,187.889 373.339,191.932 
 376.99,210.316 380.642,137.371 384.293,122.964 387.944,190.723 391.595,193.556 395.246,244.214 398.898,337.254 402.549,266.785 406.2,220.026 409.851,47.0847 
 413.502,128.916 417.154,187.789 420.805,188.16 424.456,264.95 428.107,124.146 431.759,228.307 435.41,119.376 439.061,116.328 442.712,366.066 446.363,3.93701 
 450.015,224.241 453.666,174.86 457.317,372.285 460.968,240.773 464.619,188.16 468.271,183.556 471.922,178.386 475.573,144.636 479.224,194.227 482.875,159.203 
 486.527,209.791 490.178,143.047 493.829,165.11 497.48,156.035 501.132,239.967 504.783,200.737 508.434,284.899 512.085,146.54 515.736,169.294 519.388,259.361 
 523.039,67.9479 526.69,272.337 530.341,226.331 533.992,97.8211 537.644,121.265 541.295,61.3122 544.946,127.426 548.597,151.214 552.248,175.117 555.9,199.136 
 559.551,264.34 563.202,68.8235 566.853,229.441 570.505,164.65 574.156,362.024 577.807,166.967 581.458,284.167 585.109,352.984 588.761,193.121 592.412,61.2844 
 596.063,339.219 
 "/>
 
 
 
 
 y1

In [21]:
fit

3-element Array{Any,1}:
 352.304 
  15.0627
   0.3702